[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io)

<!-- ![](https://i.imgur.com/qkg2E2D.png) -->

# AI Program

## Introduction to Estimation - The Maximum Likelihood Estimator (MLE)

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 16/09/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0004EstimationMap.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Miscellaneous
from platform import python_version
import random

# Typing
from typing import Tuple, List, Dict, Any
from numpy.typing import NDArray

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
#%matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Course Packages


In [ ]:
# Parameters

# Data
numSamples = 50
λ = 0.7

## The Data

The data is generated from the [Exponential Distribution](https://en.wikipedia.org/wiki/Exponential_distribution) model:

$$ X \sim \operatorname{Exp} \left( \lambda \right) \Rightarrow {f}_{X} \left( x ; \lambda \right) = \lambda \exp(- \lambda x) $$

Where the _deterministic_ parameter ${\lambda}$ is unknown.

* <font color='brown'>(**#**)</font> This case a closed form solution with $\hat{\lambda} = \frac{n}{\sum_{i}^{n} {x}_{i}}$.

In [ ]:
# Generate Data
# Using SciPy's Statistics module.
# SciPy's `expon` uses the scale parameter, which is the inverse of the rate parameter λ.

vX = sp.stats.expon.rvs(0, 1 / λ, size = numSamples) #<! Generating many samples

In [ ]:
# Plot the Data Histogram

hF, hA = plt.subplots(figsize = (7, 5))
hA.hist(vX, bins = 15, density = True, alpha = 0.6, color = 'g', edgecolor = 'black')
hA.set_title('Histogram of Generated Data', fontsize = 16)
hA.set_xlabel('x', fontsize = 14)
hA.set_ylabel('Density', fontsize = 14);

## The Log Likelihood Function

Given $N$ Independently Identically Distributed (IID) observations $\mathcal{D} = \left\{ {x}_{i} \right\}_{i = 1}^{N}$ from the Exponential Distribution model.

Since samples are Independent, the _Likelihood Function_ with respect to the parameter $\lambda$ is the product of densities:

$$
L \left( \lambda \mid \mathcal{D} \right) =\prod_{i = 1}^{N} \lambda e^{-\lambda x_i} = {\lambda}^{N} \exp \left( -\lambda \sum_{i=1}^{N} {x}_{i} \right)
$$

* <font color='brown'>(**#**)</font> IT is assumed all samples are non negative. Negative samples should be dropped in this model.

The _Log Likelihood Function_ is given by:

$$ \ell \left( \lambda \mid \mathcal{D} \right) = \log \left( L \left( \lambda \mid \mathcal{D} \right) \right) = N \log \left( \lambda \right) - \lambda \sum_{i = 1}^{N} {x}_{i} $$


* <font color='brown'>(**#**)</font> The Log Likelihood assists with handling large number of samples as multiplications of small numbers generates numerical issues for large ensemble.
* <font color='green'>(**@**)</font> Use the derivative to derive the closed form solution for the MLE of $\lambda$.

In [ ]:
# The Likelihood Function

def DataLogLikelihood( vX: NDArray, λ: float ) -> float:
    """
    Compute the Log-Likelihood of the data given the parameter λ.  
    Data is assumed to be drawn from an Exponential distribution.
    Samples are assumed to be independent and identically distributed (i.i.d).

    Args:
        vX (NDArray): The observed data samples.
        λ (float): The rate parameter of the Exponential distribution.

    Returns:
        float: The log-likelihood value.
    
    Remarks:
        The likelihood function for the Exponential distribution is given by:
        L(λ | vX) = ∏ (λ * exp(-λ * xi)) for xi in vX
        The log-likelihood is then:
        log L(λ | vX) = n * log(λ) - λ * ∑ xi
        where n is the number of samples and ∑ xi is the sum of the samples.
    """

    numSamples    = len(vX)
    logLikelihood = numSamples * np.log(λ) - λ * np.sum(vX)
    
    return logLikelihood

In [ ]:
# Calculate the Log Likelihood Function
# Set a range of λ values and calculate the log-likelihood for each.

vλ      = np.linspace(0.01, 10, 1000)
vLogLik = np.array([DataLogLikelihood(vX, λ) for λ in vλ])

In [ ]:
# Estimate the Parameter
# The Maximum Likelihood Estimate (MLE) of λ is the value that maximizes the log likelihood function.
# Using SciPy's optimization module to find the maximum.

oOptRes = sp.optimize.minimize_scalar(lambda λ: -DataLogLikelihood(vX, λ), bounds = (0.01, 10), method = 'bounded')
λHat = oOptRes.x

* <font color='green'>(**@**)</font> Use NumPy's [`vectorize()`](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html) to avoid the loop for the calculation.

In [ ]:
# Plot the Log Likelihood Function

hF, hA = plt.subplots(figsize = (8, 6))
hA.plot(vλ, vLogLik, color = 'b', lw = 2)
hA.set_title('Log Likelihood Function', fontsize = 16)
hA.set_xlabel('λ')
hA.set_ylabel('Log Likelihood')
hA.axvline(x = λ, color = 'r', linestyle = '--', lw = 2, label = 'True λ')
hA.axvline(x = λHat, color = 'g', linestyle = '--', lw = 2, label = 'Estimated λ (MLE)')
hA.legend();

* <font color='red'>(**?**)</font> How should the quality of the MLE estimator behave as a function of $N$? Experiment with the code to verify the answer.